In [20]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

centers = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/centers.csv')
farms = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/farms.csv')
processing = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/processing.csv')
updated_gym_data = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/updated_gym_data.csv')

1. b) Using Gurobi, what is the minimum cost of the transportation and procurement plan?

In [21]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Parameters
farm_ids = farms['Farm_ID']
processing_ids = processing['Processing_Plant_ID']
center_ids = centers['Center_ID']

farm_capacity = farms.set_index('Farm_ID')['Bio_Material_Capacity_Tons']
farm_cost = farms.set_index('Farm_ID')['Cost_Per_Ton']

# Extract transport costs from farm to processing
farm_to_processing_cost = farms.set_index('Farm_ID').filter(like="Transport_Cost_To_Plant")

processing_capacity = processing.set_index('Processing_Plant_ID')['Capacity_Tons']
processing_cost = processing.set_index('Processing_Plant_ID')['Processing_Cost_Per_Ton']

# Extract transport costs from processing to centers
processing_to_center_cost = processing.set_index('Processing_Plant_ID').filter(like="Transport_Cost_To_Center")

center_demand = centers.set_index('Center_ID')['Requested_Demand_Tons']

# Create model
model = gp.Model('Transportation_and_Procurement')

# Decision variables
x_fp = model.addVars(farm_ids, processing_ids, name="x_fp", lb=0)  # Raw material from farms to processing
x_pc = model.addVars(processing_ids, center_ids, name="x_pc", lb=0)  # Fertilizer from processing to centers

# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        x_fp[f, p] * (farm_cost[f] + farm_to_processing_cost.loc[f, f'Transport_Cost_To_Plant_{p.split("_")[-1]}'])
        for f in farm_ids for p in processing_ids
    ) +
    gp.quicksum(
        x_pc[p, c] * (processing_cost[p] + processing_to_center_cost.loc[p, f'Transport_Cost_To_Center_{c.split("_")[-1]}'])
        for p in processing_ids for c in center_ids
    ),
    GRB.MINIMIZE
)

# Constraints
# Farm capacity constraints
for f in farm_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for p in processing_ids) <= farm_capacity[f], f"FarmCapacity_{f}")

# Processing facility capacity constraints
for p in processing_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for f in farm_ids) <= processing_capacity[p], f"ProcessingCapacity_{p}")

# Center demand constraints
for c in center_ids:
    model.addConstr(gp.quicksum(x_pc[p, c] for p in processing_ids) == center_demand[c], f"CenterDemand_{c}")

# Flow balance constraints: Input to processing equals output
for p in processing_ids:
    model.addConstr(
        gp.quicksum(x_fp[f, p] for f in farm_ids) == gp.quicksum(x_pc[p, c] for c in center_ids),
        f"FlowBalance_{p}"
    )

# Solve the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    # Output the minimum cost
    print("=" * 50)
    print(f"Optimal Transportation and Procurement Cost: ${model.objVal:,.2f}")
    print("=" * 50)

    # Output detailed solution
    print("\nFarm to Processing Assignments:")
    for f in farm_ids:
        for p in processing_ids:
            if x_fp[f, p].x > 0:
                print(f"Farm {f} -> Processing {p}: {x_fp[f, p].x:.2f} tons")

    print("\nProcessing to Center Assignments:")
    for p in processing_ids:
        for c in center_ids:
            if x_pc[p, c].x > 0:
                print(f"Processing {p} -> Center {c}: {x_pc[p, c].x:.2f} tons")

else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 387 rows, 6318 columns and 17118 nonzeros
Model fingerprint: 0x3aa78cc0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   2.902000e+04   0.000000e+00      0s
     330    2.2970900e+06   0.000000e+00   0.000000e+00      0s

Solved in 330 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.297089973e+06
Optimal Transportation and Procurement Cost: $2,297,089.97

Farm to Processing Assignments:
Farm Farm_4 -> Processing Plant_9: 367.00 tons
Farm Farm_5 -> Processing Plant_6: 499.00 tons
Farm Farm_6 -> Pro

1. c) If the processing plants of the raw material are restricted to only send fertilizer to home centers
within the same region of the US, what is the optimal cost?

In [22]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Parameters
farm_ids = farms['Farm_ID']
processing_ids = processing['Processing_Plant_ID']
center_ids = centers['Center_ID']

farm_capacity = farms.set_index('Farm_ID')['Bio_Material_Capacity_Tons']
farm_cost = farms.set_index('Farm_ID')['Cost_Per_Ton']

# Extract transport costs from farm to processing
farm_to_processing_cost = farms.set_index('Farm_ID').filter(like="Transport_Cost_To_Plant")

processing_capacity = processing.set_index('Processing_Plant_ID')['Capacity_Tons']
processing_cost = processing.set_index('Processing_Plant_ID')['Processing_Cost_Per_Ton']

# Extract transport costs from processing to centers
processing_to_center_cost = processing.set_index('Processing_Plant_ID').filter(like="Transport_Cost_To_Center")

center_demand = centers.set_index('Center_ID')['Requested_Demand_Tons']

# Extract regions
processing_regions = processing.set_index('Processing_Plant_ID')['Region']
center_regions = centers.set_index('Center_ID')['Region']

# Create model
model = gp.Model('Transportation_and_Procurement_Regional')

# Decision variables
x_fp = model.addVars(farm_ids, processing_ids, name="x_fp", lb=0)  # Raw material from farms to processing
x_pc = model.addVars(processing_ids, center_ids, name="x_pc", lb=0)  # Fertilizer from processing to centers

# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        x_fp[f, p] * (farm_cost[f] + farm_to_processing_cost.loc[f, f'Transport_Cost_To_Plant_{p.split("_")[-1]}'])
        for f in farm_ids for p in processing_ids
    ) +
    gp.quicksum(
        x_pc[p, c] * (processing_cost[p] + processing_to_center_cost.loc[p, f'Transport_Cost_To_Center_{c.split("_")[-1]}'])
        for p in processing_ids for c in center_ids
    ),
    GRB.MINIMIZE
)

# Constraints
# Farm capacity constraints
for f in farm_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for p in processing_ids) <= farm_capacity[f], f"FarmCapacity_{f}")

# Processing facility capacity constraints
for p in processing_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for f in farm_ids) <= processing_capacity[p], f"ProcessingCapacity_{p}")

# Center demand constraints
for c in center_ids:
    model.addConstr(gp.quicksum(x_pc[p, c] for p in processing_ids) == center_demand[c], f"CenterDemand_{c}")

# Flow balance constraints: Input to processing equals output
for p in processing_ids:
    model.addConstr(
        gp.quicksum(x_fp[f, p] for f in farm_ids) == gp.quicksum(x_pc[p, c] for c in center_ids),
        f"FlowBalance_{p}"
    )

# Regional constraints: Processing plants can only send to centers in the same region
for p in processing_ids:
    for c in center_ids:
        if processing_regions[p] != center_regions[c]:
            model.addConstr(x_pc[p, c] == 0, f"RegionalConstraint_{p}_{c}")

# Solve the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    # Output the minimum cost
    print("=" * 50)
    print(f"Optimal Transportation and Procurement Cost (Regional): ${model.objVal:,.2f}")
    print("=" * 50)

    # Output detailed solution
    print("\nFarm to Processing Assignments:")
    for f in farm_ids:
        for p in processing_ids:
            if x_fp[f, p].x > 0:
                print(f"Farm {f} -> Processing {p}: {x_fp[f, p].x:.2f} tons")

    print("\nProcessing to Center Assignments:")
    for p in processing_ids:
        for c in center_ids:
            if x_pc[p, c].x > 0:
                print(f"Processing {p} -> Center {c}: {x_pc[p, c].x:.2f} tons")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1761 rows, 6318 columns and 18492 nonzeros
Model fingerprint: 0x9c296277
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 1374 rows and 1374 columns
Presolve time: 0.01s
Presolved: 387 rows, 4944 columns, 14370 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2479485e+05   3.627500e+03   0.000000e+00      0s
     320    2.3234885e+06   0.000000e+00   0.000000e+00      0s

Solved in 320 iterations and 0.01 seconds (0.01 work units)
Optimal objective  2.323488497e+06
Optimal Transportation and Procurement Cost (Regional): $2,323,488.50

Farm to Processing Assignments:
Farm Farm_4 -> Processing Plant_9: 367.00 tons
Farm Farm

1. d) If only the highest quality raw material (i.e., levels 3 and 4) is sourced from farms to make fertilizer, what is the optimal cost?

In [24]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Parameters
farm_ids = farms['Farm_ID']
processing_ids = processing['Processing_Plant_ID']
center_ids = centers['Center_ID']

farm_capacity = farms.set_index('Farm_ID')['Bio_Material_Capacity_Tons']
farm_cost = farms.set_index('Farm_ID')['Cost_Per_Ton']
farm_quality = farms.set_index('Farm_ID')['Quality']  # Add quality column

# Extract transport costs from farm to processing
farm_to_processing_cost = farms.set_index('Farm_ID').filter(like="Transport_Cost_To_Plant")

processing_capacity = processing.set_index('Processing_Plant_ID')['Capacity_Tons']
processing_cost = processing.set_index('Processing_Plant_ID')['Processing_Cost_Per_Ton']

# Extract transport costs from processing to centers
processing_to_center_cost = processing.set_index('Processing_Plant_ID').filter(like="Transport_Cost_To_Center")

center_demand = centers.set_index('Center_ID')['Requested_Demand_Tons']

# Create model
model = gp.Model('Transportation_and_Procurement_HighQuality')

# Decision variables
x_fp = model.addVars(farm_ids, processing_ids, name="x_fp", lb=0)  # Raw material from farms to processing
x_pc = model.addVars(processing_ids, center_ids, name="x_pc", lb=0)  # Fertilizer from processing to centers

# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        x_fp[f, p] * (farm_cost[f] + farm_to_processing_cost.loc[f, f'Transport_Cost_To_Plant_{p.split("_")[-1]}'])
        for f in farm_ids for p in processing_ids
    ) +
    gp.quicksum(
        x_pc[p, c] * (processing_cost[p] + processing_to_center_cost.loc[p, f'Transport_Cost_To_Center_{c.split("_")[-1]}'])
        for p in processing_ids for c in center_ids
    ),
    GRB.MINIMIZE
)

# Constraints
# Farm capacity constraints
for f in farm_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for p in processing_ids) <= farm_capacity[f], f"FarmCapacity_{f}")

# Processing facility capacity constraints
for p in processing_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for f in farm_ids) <= processing_capacity[p], f"ProcessingCapacity_{p}")

# Center demand constraints
for c in center_ids:
    model.addConstr(gp.quicksum(x_pc[p, c] for p in processing_ids) == center_demand[c], f"CenterDemand_{c}")

# Flow balance constraints: Input to processing equals output
for p in processing_ids:
    model.addConstr(
        gp.quicksum(x_fp[f, p] for f in farm_ids) == gp.quicksum(x_pc[p, c] for c in center_ids),
        f"FlowBalance_{p}"
    )

# Quality-based constraints: Only farms with quality levels 3 or 4 can supply raw materials
for f in farm_ids:
    if farm_quality[f] < 3:  # Exclude farms with quality less than 3
        for p in processing_ids:
            model.addConstr(x_fp[f, p] == 0, f"QualityConstraint_{f}")

# Solve the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    # Output the minimum cost
    print("=" * 50)
    print(f"Optimal Transportation and Procurement Cost (High Quality): ${model.objVal:,.2f}")
    print("=" * 50)

    # Output detailed solution
    print("\nFarm to Processing Assignments:")
    for f in farm_ids:
        for p in processing_ids:
            if x_fp[f, p].x > 0:
                print(f"Farm {f} -> Processing {p}: {x_fp[f, p].x:.2f} tons")

    print("\nProcessing to Center Assignments:")
    for p in processing_ids:
        for c in center_ids:
            if x_pc[p, c].x > 0:
                print(f"Processing {p} -> Center {c}: {x_pc[p, c].x:.2f} tons")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3519 rows, 6318 columns and 20250 nonzeros
Model fingerprint: 0xa45bebf3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 3306 rows and 3132 columns
Presolve time: 0.01s
Presolved: 213 rows, 3186 columns, 7722 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   3.627500e+03   0.000000e+00      0s
     253    5.7129767e+06   0.000000e+00   0.000000e+00      0s

Solved in 253 iterations and 0.01 seconds (0.01 work units)
Optimal objective  5.712976673e+06
Optimal Transportation and Procurement Cost (High Quality): $5,712,976.67

Farm to Processing Assignments:
Farm Farm_3 -> Processing Plant_10: 516.00 tons
Farm 

1. e) If each facility is limited to processing no more than 3% of all raw material sourced from farms (as a sourcing risk mitigation measure), what is the optimal cost? Alternatively, if a production facility is limited to supplying no more than 50% of all fertilizer to a single home center (as a supply risk mitigation measure), what is the optimal cost?


In [25]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Parameters
farm_ids = farms['Farm_ID']
processing_ids = processing['Processing_Plant_ID']
center_ids = centers['Center_ID']

farm_capacity = farms.set_index('Farm_ID')['Bio_Material_Capacity_Tons']
farm_cost = farms.set_index('Farm_ID')['Cost_Per_Ton']

# Extract transport costs from farm to processing
farm_to_processing_cost = farms.set_index('Farm_ID').filter(like="Transport_Cost_To_Plant")

processing_capacity = processing.set_index('Processing_Plant_ID')['Capacity_Tons']
processing_cost = processing.set_index('Processing_Plant_ID')['Processing_Cost_Per_Ton']

# Extract transport costs from processing to centers
processing_to_center_cost = processing.set_index('Processing_Plant_ID').filter(like="Transport_Cost_To_Center")

center_demand = centers.set_index('Center_ID')['Requested_Demand_Tons']

# Total raw material sourced from farms
total_raw_material = farm_capacity.sum()

# Create model
model = gp.Model('Transportation_and_Procurement_RiskMitigation')

# Decision variables
x_fp = model.addVars(farm_ids, processing_ids, name="x_fp", lb=0)  # Raw material from farms to processing
x_pc = model.addVars(processing_ids, center_ids, name="x_pc", lb=0)  # Fertilizer from processing to centers

# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        x_fp[f, p] * (farm_cost[f] + farm_to_processing_cost.loc[f, f'Transport_Cost_To_Plant_{p.split("_")[-1]}'])
        for f in farm_ids for p in processing_ids
    ) +
    gp.quicksum(
        x_pc[p, c] * (processing_cost[p] + processing_to_center_cost.loc[p, f'Transport_Cost_To_Center_{c.split("_")[-1]}'])
        for p in processing_ids for c in center_ids
    ),
    GRB.MINIMIZE
)

# Constraints
# Farm capacity constraints
for f in farm_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for p in processing_ids) <= farm_capacity[f], f"FarmCapacity_{f}")

# Processing facility capacity constraints
for p in processing_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for f in farm_ids) <= processing_capacity[p], f"ProcessingCapacity_{p}")

# Center demand constraints
for c in center_ids:
    model.addConstr(gp.quicksum(x_pc[p, c] for p in processing_ids) == center_demand[c], f"CenterDemand_{c}")

# Flow balance constraints: Input to processing equals output
for p in processing_ids:
    model.addConstr(
        gp.quicksum(x_fp[f, p] for f in farm_ids) == gp.quicksum(x_pc[p, c] for c in center_ids),
        f"FlowBalance_{p}"
    )

# 1. Facility Processing Constraint: No more than 3% of total raw material sourced
for p in processing_ids:
    model.addConstr(gp.quicksum(x_fp[f, p] for f in farm_ids) <= 0.03 * total_raw_material, f"MaxProcessing_{p}")

# 2. Facility Supply Constraint: No more than 50% of fertilizer to a single home center
for p in processing_ids:
    for c in center_ids:
        model.addConstr(x_pc[p, c] <= 0.5 * gp.quicksum(x_pc[p, cc] for cc in center_ids), f"MaxSupply_{p}_{c}")

# Solve the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    # Output the minimum cost
    print("=" * 50)
    print(f"Optimal Transportation and Procurement Cost (Risk Mitigation): ${model.objVal:,.2f}")
    print("=" * 50)

    # Output detailed solution
    print("\nFarm to Processing Assignments:")
    for f in farm_ids:
        for p in processing_ids:
            if x_fp[f, p].x > 0:
                print(f"Farm {f} -> Processing {p}: {x_fp[f, p].x:.2f} tons")

    print("\nProcessing to Center Assignments:")
    for p in processing_ids:
        for c in center_ids:
            if x_pc[p, c].x > 0:
                print(f"Processing {p} -> Center {c}: {x_pc[p, c].x:.2f} tons")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2241 rows, 6318 columns and 208872 nonzeros
Model fingerprint: 0xd8387f79
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.03s
Presolved: 2223 rows, 6318 columns, 204390 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.05 seconds (0.15 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
     850    2.3118253e+06   0.000000e+00   0.000000e+00      0s

Solved in 850 iterations and 0.06 seconds (0.18 

1. f) Four options were evaluated to understand how changes to the supply chain impacted cost, i.e., see parts (c) through (e). Which of these options (or multiple) are financially defensible, and why? What is the optimal cost when you implement all of the defensible options together?

1. g) While implementing all of the defensible options together incurs a higher cost as compared to the original system, it may still represent a strong business decision. How would you concisely defend the implementation of all of the defensible options to management?

1. h) The supply chain network has a limited capacity for risk mitigation. To see this, when implementing all of the defensible options from part (f), at what value (to the nearest tenth of a percent) does the model become infeasible when reducing the sourcing risk mitigation percentage from the value given in part (e) of 3%? What is the managerial interpretation of this result, and what are the implications for managing supply chain risk?